# LDA2Vec
Adapted from tensorflow implementation: 

(Current) https://github.com/nateraw/Lda2vec-Tensorflow

(Old) https://github.com/meereeum/lda2vec-tf

(Original) https://github.com/cemoody/lda2vec

In [1]:
import pandas as pd
import numpy as np
import pyLDAvis
import pickle

In [2]:
import sys
sys.path.append('./Lda2vec-Tensorflow-master')

In [3]:
from lda2vec import *

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0Metal device set to: Apple M1



2021-11-08 13:19:28.474691: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-08 13:19:28.475078: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Find Ending Index of Documents for Each Year

In [4]:
data = pd.read_csv("papers.csv")

In [5]:
data = data.sort_values(by=["year"], kind='mergesort')

In [6]:
years = list(data["year"].unique())
counts = list(data["year"].value_counts())[::-1]
for i in range(1,len(counts)):
  counts[i] += counts[i-1]
year_index = {years[i]:counts[i] for i in range(len(years))}

In [7]:
year_index

{1987: 90,
 1988: 184,
 1989: 285,
 1990: 412,
 1991: 552,
 1992: 695,
 1993: 839,
 1994: 989,
 1995: 1139,
 1996: 1290,
 1997: 1442,
 1998: 1594,
 1999: 1746,
 2000: 1904,
 2001: 2101,
 2002: 2299,
 2003: 2503,
 2004: 2710,
 2005: 2917,
 2006: 3124,
 2007: 3341,
 2008: 3591,
 2009: 3853,
 2010: 4145,
 2011: 4451,
 2012: 4811,
 2013: 5179,
 2014: 5582,
 2015: 5993,
 2016: 6562,
 2017: 7241}

## Preprocessing

$\tt cleaned.txt$ is a txt file with all NIPS papers with all stop words and words with $\rm{length} \leq 3$ removed.

Preprocess incorporates tokenization (splitting sentences into words), creating a vocabulary to save mappings from tokens to integer indices, and generating skip-grams.

In [8]:
df = pd.read_csv("cleaned.txt")
df = df[df["stop_removed_paper_text"].notnull()]

In [9]:
df = df[4451:4551] # Use only 2011 papers

In [10]:
# Initialize a preprocessor
P = nlppipe.Preprocessor(df, "stop_removed_paper_text", max_features=30000, maxlen=10000, min_count=30)

In [11]:
# Run the preprocessing on your dataframe
P.preprocess()


---------- Tokenizing Texts ----------


100it [00:35,  2.78it/s]


Removing 12703 low frequency tokens out of 14216 total tokens

---------- Getting Skipgrams ----------


100it [00:01, 50.56it/s]


In [12]:
# Should we load pretrained embeddings from file
load_embeds = True

# Load embeddings from file if we choose to do so
if load_embeds:
    # Load embedding matrix from file path - change path to where you saved them
    embedding_matrix = P.load_glove("glove.6B.300d.txt")
else:
    embedding_matrix = None

/Users/keanl/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# Save data to data_dir
P.save_data("clean_data11", embedding_matrix=embedding_matrix)

## Using the LDA2Vec Model

Using the LDA2Vec model on preprocessed data.

In [5]:
# Path to preprocessed data
data_path  = "clean_data11"
# Whether or not to load saved embeddings file
load_embeds = True

# Load data from files
(idx_to_word, word_to_idx, freqs, pivot_ids,
 target_ids, doc_ids, embed_matrix) = utils.load_preprocessed_data(data_path, load_embed_matrix=load_embeds)

In [6]:
# Number of unique documents
num_docs = doc_ids.max() + 1
# Number of unique words in vocabulary (int)
vocab_size = len(freqs)
# Embed layer dimension size
# If not loading embeds, change 128 to whatever size you want.
embed_size = embed_matrix.shape[1] if load_embeds else 128
# Number of topics to cluster into
num_topics = 20
# Amount of iterations over entire dataset
num_epochs = 50
# Batch size - Increase/decrease depending on memory usage
batch_size = 4096
# Epoch that we want to "switch off" regularization
switch_loss_epoch = 0
# Pretrained embeddings value
pretrained_embeddings = embed_matrix if load_embeds else None
# If True, save logdir, otherwise don't
save_graph = False

In [7]:
# Initialize the model
m = model(num_docs,
          vocab_size,
          num_topics,
          embedding_size=embed_size,
          pretrained_embeddings=pretrained_embeddings,
          freqs=freqs,
          batch_size = batch_size,
          save_graph_def=save_graph)

2021-11-08 13:19:32.885822: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-08 13:19:32.885883: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-11-08 13:19:33.054677: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-08 13:19:33.055450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
# Train the model
m.train(pivot_ids,
        target_ids,
        doc_ids,
        len(pivot_ids),
        num_epochs,
        idx_to_word=idx_to_word,
        switch_loss_epoch=switch_loss_epoch)

2021-11-08 13:19:33.711420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-08 13:19:33.728528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-08 13:19:33.732312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.



EPOCH: 1


2021-11-08 13:19:41.331494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 7489.628 w2v 13.490599 lda 7476.137 reg 63.779526

EPOCH: 2


2021-11-08 13:19:49.211522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

LOSS 2703.043 w2v 4.131691 lda 2698.9114 reg 82.260635

EPOCH: 29


2021-11-08 13:23:14.493911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 2701.4995 w2v 4.3341393 lda 2697.1653 reg 82.35562

EPOCH: 30


2021-11-08 13:23:21.735277: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-08 13:23:21.751219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


LOSS 2700.5864 w2v 4.6171403 lda 2695.9692 reg 82.429016
---------Closest 10 words to given indexes----------
Topic 0 : strati, cuboid, stratum, hyper, cation, strata, queries, mwem, faces, variations
Topic 1 : nogo, hashing, threshold, trial, bias, trials, agent, perceptual, response, decision
Topic 2 : blei, mmsb, collapsed, communities, variational, mixtures, subsampling, jordan, mmtm, dirichlet
Topic 3 : transmission, netrate, cascades, kernelcascade, diffusion, influence, submodular, corrlda, edges, edge
Topic 4 : active, shamo, instances, labeling, batch, pseudo, costs, asymptotically, false, pool
Topic 5 : chernoff, utility, optimizing, aware, tradeoff, ppca, differentially, trajectory, reinforcement, coaching
Topic 6 : pooling, filters, selective, eigenvector, seed, ista, unlabeled, cell, alternating, supervised
Topic 7 : slice, hidden, formulas, discriminative, shamo, bxyz, mcmc, logic, fields, marginals
Topic 8 : classi, cation, calibrated, surrogate, multiclass, procedures, 

## Get Word and Topic Embeddings

Visualize topics

In [18]:
idx_to_word

{1: 'learning',
 2: 'model',
 3: 'algorithm',
 4: 'data',
 5: 'function',
 6: 'time',
 7: 'models',
 8: 'problem',
 9: 'distribution',
 10: 'number',
 11: 'method',
 12: 'given',
 13: 'matrix',
 14: 'results',
 15: 'figure',
 16: 'methods',
 17: 'bound',
 18: 'error',
 19: 'based',
 20: 'algorithms',
 21: 'optimization',
 22: 'information',
 23: 'linear',
 24: 'space',
 25: 'gaussian',
 26: 'loss',
 27: 'variables',
 28: 'probability',
 29: 'random',
 30: 'section',
 31: 'convex',
 32: 'theorem',
 33: 'inference',
 34: 'approach',
 35: 'case',
 36: 'training',
 37: 'optimal',
 38: 'vector',
 39: 'regression',
 40: 'sample',
 41: 'different',
 42: 'following',
 43: 'large',
 44: 'sampling',
 45: 'task',
 46: 'parameter',
 47: 'performance',
 48: 'machine',
 49: 'prior',
 50: 'functions',
 51: 'network',
 52: 'classification',
 53: 'work',
 54: 'consider',
 55: 'test',
 56: 'value',
 57: 'analysis',
 58: 'problems',
 59: 'likelihood',
 60: 'mean',
 61: 'distributions',
 62: 'feature',
 6

In [9]:
doc_embed = m.sesh.run(m.mixture.doc_embedding)

2021-11-08 09:22:55.507931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [10]:
topic_embed = m.sesh.run(m.mixture.topic_embedding)

2021-11-08 09:22:56.071969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [11]:
word_embed = m.sesh.run(m.w_embed.embedding)

2021-11-08 09:22:56.587139: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [12]:
# Extract all unique words in order of index 0-vocab_size
vocabulary = []
for k,v in idx_to_word.items():
    vocabulary.append(v)

In [13]:
doc_lengths = np.load("clean_data11" + "/doc_lengths.npy")

In [14]:
vis_data = utils.prepare_topics(doc_embed, topic_embed, word_embed, np.array(vocabulary), doc_lengths=doc_lengths,
                              term_frequency=freqs, normalize=True)

In [15]:
prepared_vis_data = pyLDAvis.prepare(**vis_data)

/Users/keanl/miniforge3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/keanl/miniforge3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/keanl/miniforge3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/keanl/miniforge3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/keanl/miniforge3/lib/python3.9/site-packages/pa

In [16]:
pyLDAvis.display(prepared_vis_data)

In [27]:
np.save("doc_embed11", doc_embed)

In [28]:
np.save("word_embed11", word_embed)

In [29]:
np.save("topic_embed11", topic_embed)

In [97]:
def closest(embed_idxs, embed_type):
    if embed_type == 0:  # Topics
        norm_embeds = topic_embed / np.linalg.norm(topic_embed, axis=1).reshape((topic_embed.shape[0],1))
        embed_vec = topic_embed[embed_idxs[2]] - topic_embed[embed_idxs[0]] + topic_embed[embed_idxs[1]]
        embed_norm = (embed_vec) / np.linalg.norm(embed_vec)
    else:  # Words
        norm_embeds = word_embed / np.linalg.norm(word_embed, axis=1).reshape((word_embed.shape[0],1))
        embed_vec = word_embed[embed_idxs[2]] - word_embed[embed_idxs[0]] + word_embed[embed_idxs[1]]
        embed_norm = (embed_vec) / np.linalg.norm(embed_vec)
    cos_sim = np.dot(norm_embeds, embed_norm)
    sort = (-cos_sim).argsort()
    for i in range(3): sort=sort[sort != embed_idxs[i]]
    return sort[0]

In [98]:
def cosadd(embed1, embed2, embed3, embed4, embed_type):
    if embed_type == 0:
        norm_embed1 = topic_embed[embed1] / np.linalg.norm(topic_embed[embed1])
        norm_embed2 = topic_embed[embed2] / np.linalg.norm(topic_embed[embed2])
        norm_embed3 = topic_embed[embed3] / np.linalg.norm(topic_embed[embed3])
        norm_embed4 = topic_embed[embed4] / np.linalg.norm(topic_embed[embed4])
    else:
        norm_embed1 = word_embed[embed1] / np.linalg.norm(word_embed[embed1])
        norm_embed2 = word_embed[embed2] / np.linalg.norm(word_embed[embed2])
        norm_embed3 = word_embed[embed3] / np.linalg.norm(word_embed[embed3])
        norm_embed4 = word_embed[embed4] / np.linalg.norm(word_embed[embed4])
    return np.dot(norm_embed4, norm_embed3) - np.dot(norm_embed4, norm_embed1) + np.dot(norm_embed4, norm_embed2)

In [ ]:
from itertools import *
from math import comb
combine = list(combinations(list(range(word_embed.shape[0])), 3))
score = np.zeros(comb(word_embed.shape[0],3))
for i in range(score.shape[0]):
    (embed_idx1, embed_idx2, embed_idx3) = combine[i]
    score[i] = cosadd(embed_idx1, embed_idx2, embed_idx3, closest([embed_idx1, embed_idx2, embed_idx3], 1), 1)
top_analogies = [list(combine[idx])+[closest([combine[idx][0], combine[idx][1], combine[idx][2]], 1)] for idx in (-score).argsort()]
top_scores = [score[idx] for idx in (-score).argsort()]

In [126]:
top_analogies[:5]

[[0, 7, 15, 6],
 [2, 15, 16, 14],
 [10, 15, 16, 14],
 [0, 15, 17, 6],
 [2, 9, 16, 14]]

In [115]:
top_scores[:5]

[1.157698631286621,
 1.1547048091888428,
 1.1386322975158691,
 1.1078553199768066,
 1.1042547225952148]

In [101]:
cosadd(10, 14, 33, 167, 1)

0.53526247

In [127]:
word_embed.shape

(1513, 300)

In [ ]:
m.sesh.get_k_closest([], in_type='word', vs_type='word', k=10, idx_to_word=None, verbose=False):